<a href="https://colab.research.google.com/github/EveJL/TopicModeling/blob/main/BERTopic_(semi)_supervised_topic_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tutorial** - (semi)-supervised topic modeling
(last updated 11-09-2022)

In this tutorial, we will be looking at a new feature of BERTopic, namely (semi)-supervised topic modeling! This allows us to steer the dimensionality reduction of the embeddings into a space that closely follows any labels you might already have. 

## Semi-supervised modeling
(semi)-supervised topic modeling is a class of methods that allows the user to perform topic modeling with previously defined labels. This might help nudge the model towards specific topics or classes for which you have labels. 

<br>

<img src="https://raw.githubusercontent.com/MaartenGr/BERTopic/master/images/logo.png" width="40%">

# Enabling the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

[Reference](https://colab.research.google.com/notebooks/gpu.ipynb)

# Installing BERTopic

We start by installing BERTopic from PyPi:

In [1]:
%%capture
!pip install bertopic

## Restart the Notebook
After installing BERTopic, some packages that were already loaded were updated and in order to correctly use them, we should now restart the notebook.

From the Menu:

Runtime → Restart Runtime

# **Data**
For this example, we use the popular 20 Newsgroups dataset which contains roughly 18000 newsgroups posts that each is assigned to one of 20 topics:

In [2]:
import pandas as pd
from bertopic import BERTopic
from sklearn.datasets import fetch_20newsgroups

data = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))
docs = data["data"]
targets = data["target"]
target_names = data["target_names"]
classes = [data["target_names"][i] for i in data["target"]]

In [20]:
type(docs)

list

In [21]:
len(docs)

18846

Text is not pre-processed. 

In [22]:
docs[0]

"\n\nI am sure some bashers of Pens fans are pretty confused about the lack\nof any kind of posts about the recent Pens massacre of the Devils. Actually,\nI am  bit puzzled too and a bit relieved. However, I am going to put an end\nto non-PIttsburghers' relief with a bit of praise for the Pens. Man, they\nare killing those Devils worse than I thought. Jagr just showed you why\nhe is much better than his regular season stats. He is also a lot\nfo fun to watch in the playoffs. Bowman should let JAgr have a lot of\nfun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final\nregular season game.          PENS RULE!!!\n\n"

Each document can be put into one of the following categories:

In [3]:
target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

# **(semi)-Supervised modeling**


## Basic Model
Before we start with semi-supervised modeling, let us first take a look at the output of the basic model.

In [4]:
topic_model = BERTopic(verbose=True)
topics, _ = topic_model.fit_transform(docs)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

Batches:   0%|          | 0/589 [00:00<?, ?it/s]

2022-10-06 15:38:32,274 - BERTopic - Transformed documents to Embeddings
2022-10-06 15:39:04,598 - BERTopic - Reduced dimensionality
2022-10-06 15:39:06,108 - BERTopic - Clustered reduced embeddings


In [5]:
topic_model.get_topic_info().head(10)

,Topic,Count,Name
0,-1,6754,-1_to_the_is_and
1,0,1840,0_game_team_games_he
2,1,580,1_key_clipper_chip_encryption
3,2,527,2_ites_cheek_yep_huh
4,3,391,3_you_your_post_jim
5,4,360,4_israel_israeli_arab_arabs
6,5,337,5_car_engine_cars_ford
7,6,278,6_bike_riding_ride_my
8,7,195,7_jpeg_image_gif_images
9,8,161,8_bike_tires_bikes_tire


The topics that were created mostly make sense. There are some clearly defined topics such as "nasa, orbit, spacecraft, moon" but also some topics that seem mostly derived from other topics. We can visualize this by extracting the topic representations per class and see if our unsupervised model closely resembles this. 

**NOTE**: You can **hover** over the bars to see the representation per class!!

In [6]:
topics_per_class = topic_model.topics_per_class(docs, classes=classes)
fig_unsupervised = topic_model.visualize_topics_per_class(topics_per_class, top_n_topics=10)
fig_unsupervised

20it [00:06,  3.11it/s]


The results do seem promising. Topics like "nasa, space, etc" seem to be clearly related to sci.space, but some topics were created that span many categories. For example, we expect the topic "bike, bikes, etc"  to only appear in rec.motorcycles.  

## Semi-supervised
In the example above you might notice that some topics were somewhat smushed together. What we would like to see is a clear separation between those topics. Fortunately, we have to labels and can use them to improve the model. 

Since we are not interested in any other topics, this method is called **semi-supervised topic modeling, which means that we have the labels of some documents but not all.** *(Exactly what I need!)*

For this example let's say we only have the labels of all computer-related categories:

In [7]:
 labels_to_add = ['comp.graphics', 'comp.os.ms-windows.misc',
                  'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware',
                  'comp.windows.x',]
 indices = [target_names.index(label) for label in labels_to_add]
 new_labels = [label if label in indices else -1 for label in targets]

When generating our new labels it is important to mark unknown classes as **-1**. Next, we use those newly constructed labels to again run BERTopic:

In [8]:
topic_model = BERTopic(verbose=True)
topics, _ = topic_model.fit_transform(docs, y=new_labels)

Batches:   0%|          | 0/589 [00:00<?, ?it/s]

2022-10-06 15:39:34,454 - BERTopic - Transformed documents to Embeddings
2022-10-06 15:39:48,704 - BERTopic - Reduced dimensionality
2022-10-06 15:39:49,479 - BERTopic - Clustered reduced embeddings


In [9]:
topic_model.get_topic_info().head(10)

,Topic,Count,Name
0,-1,6125,-1_to_of_the_and
1,0,1837,0_game_team_he_games
2,1,876,1_window_server_motif_widget
3,2,586,2_key_clipper_chip_encryption
4,3,529,3_ites_cheek_yep_dir
5,4,465,4_israel_israeli_jews_arab
6,5,394,5_you_your_post_jim
7,6,273,6_bike_ride_riding_my
8,7,186,7_car_cars_mustang_ford
9,8,160,8_gay_sex_homosexual_sexual


Finally, we can again extract the topics per class to see if our semi-supervised approach had some effect:

In [10]:
topics_per_class = topic_model.topics_per_class(docs, classes=classes)
fig_semi_supervised = topic_model.visualize_topics_per_class(topics_per_class, top_n_topics=10, width=900)
fig_semi_supervised

20it [00:06,  3.10it/s]


We can clearly see that many more topics about computers were created and that the seperation between those topics are solid. This indicates that even if you do not have all the labels, you can definitely improve the model!

However, there are still some clusters that could be improved with the labels that we have. 

## Supervised

Finally, we are going to be using all labels. These labels help BERTopic understand where most clusters can be found. However, this does not mean that it will only find the 20 clusters that we have defined. If there are sub-clusters to be found, then there is a good chance BERTopic will find them! 

In [11]:
topic_model = BERTopic(verbose=True)
topics, _ = topic_model.fit_transform(docs, y=targets)

Batches:   0%|          | 0/589 [00:00<?, ?it/s]

2022-10-06 15:40:17,083 - BERTopic - Transformed documents to Embeddings
2022-10-06 15:40:28,643 - BERTopic - Reduced dimensionality
2022-10-06 15:40:29,350 - BERTopic - Clustered reduced embeddings


In [12]:
topic_model.get_topic_info().head(10)

,Topic,Count,Name
0,-1,3830,-1_the_of_to_and
1,0,932,0_bike_dod_ride_my
2,1,931,1_space_launch_nasa_orbit
3,2,911,2_he_game_year_baseball
4,3,906,3_windows_dos_file_files
5,4,900,4_window_server_motif_widget
6,5,863,5_mac_apple_drive_with
7,6,857,6_gun_guns_firearms_they
8,7,814,7_key_encryption_clipper_chip
9,8,566,8_israel_israeli_arab_jews


Not only do we see a nice seperation of the topics, there are significantly less outliers which shows that BERTopic has improved in connecting the documents to topics. 

Let's see the results by again visualizing the topic representation per class:

In [13]:
topics_per_class = topic_model.topics_per_class(docs, classes=classes)
fig_supervised = topic_model.visualize_topics_per_class(topics_per_class, top_n_topics=10, width=900)
fig_supervised

20it [00:06,  3.24it/s]


Now that we have used all labels, BERTopic seems to closely match our pre-defined labels. Moreover, it still allows to discover topics that were not previously defined. Thus, you can use this method to find unknown topics in pre-defined topics!